In [1]:
import pandas as pd
from common import Data

In [2]:
df = Data.get_nwsl_goals(season="2023")
df["opponent"] = df.apply(
    lambda x: x["home"] if x["home"] != x["team"] else x["away"], axis=1
)
df

,season,date,home,away,player,team,assist,period,minute,second,x,y,opponent
2161,2023,2023-03-25,NC,KC,Mille Gejl Jensen,NC,Narumi Miura,1,22,24,70.4,63.8,KC
2162,2023,2023-03-25,SD,CHI,Yuki Ogimi Nagasato,CHI,NaN,1,17,41,97.6,47.4,SD
2163,2023,2023-03-25,SD,CHI,Amirah Ali,SD,NaN,1,21,34,98.3,48.4,CHI
2164,2023,2023-03-25,SD,CHI,Jaedyn Shaw,SD,NaN,1,31,7,81.6,50.4,CHI
2165,2023,2023-03-25,SD,CHI,Mallory Pugh Swanson,CHI,NaN,1,48,18,88.5,50.0,SD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,2023,2023-10-15,SD,LOU,Alex Morgan,SD,Madison Pogarch,2,46,19,84.3,62.7,LOU
2499,2023,2023-10-15,WAS,NC,Tyler Lussi,NC,Manaka Matsukubo,1,24,43,95.0,54.3,WAS
2500,2023,2023-10-20,RGN,LA,Veronica Latsko,RGN,Phoebe McClernon,2,86,32,91.0,47.0,LA
2501,2023,2023-10-22,NC,NJY,Delanie Sheehan,NJY,Bruninha,1,44,56,89.9,31.0,NC


In [3]:
cols = {
    "0-15": (1, 0, 15),
    "15-30": (1, 15, 30),
    "30-45+": (1, 30, 90),
    "45-60": (2, 45, 60),
    "60-75": (2, 60, 75), 
    "75-90+": (2, 75, 135), 
}

for col in cols:
    half, lower, upper = cols[col]
    df[col] = (df["period"] == half) & (df["minute"] >= lower) & (df["minute"] < upper)
    
df = df.drop(columns=["period", "minute", "x", "y", "second"])
    
df

,season,date,home,away,player,team,assist,opponent,0-15,15-30,30-45+,45-60,60-75,75-90+
2161,2023,2023-03-25,NC,KC,Mille Gejl Jensen,NC,Narumi Miura,KC,False,True,False,False,False,False
2162,2023,2023-03-25,SD,CHI,Yuki Ogimi Nagasato,CHI,NaN,SD,False,True,False,False,False,False
2163,2023,2023-03-25,SD,CHI,Amirah Ali,SD,NaN,CHI,False,True,False,False,False,False
2164,2023,2023-03-25,SD,CHI,Jaedyn Shaw,SD,NaN,CHI,False,False,True,False,False,False
2165,2023,2023-03-25,SD,CHI,Mallory Pugh Swanson,CHI,NaN,SD,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,2023,2023-10-15,SD,LOU,Alex Morgan,SD,Madison Pogarch,LOU,False,False,False,True,False,False
2499,2023,2023-10-15,WAS,NC,Tyler Lussi,NC,Manaka Matsukubo,WAS,False,True,False,False,False,False
2500,2023,2023-10-20,RGN,LA,Veronica Latsko,RGN,Phoebe McClernon,LA,False,False,False,False,False,True
2501,2023,2023-10-22,NC,NJY,Delanie Sheehan,NJY,Bruninha,NC,False,False,True,False,False,False


In [4]:
scored = df.groupby("team").sum()
scored.to_csv("data/goals_scored_2023.csv")
scored

,0-15,15-30,30-45+,45-60,60-75,75-90+
team,,,,,,
CHI,1,5,6,6,8,1
HOU,2,2,5,4,3,1
KC,5,7,4,5,7,4
LA,3,2,8,3,6,7
LOU,3,4,4,5,3,4
NC,3,5,8,6,3,3
NJY,5,2,4,4,7,6
ORL,3,10,3,4,5,3
POR,8,9,6,5,8,8


In [5]:
scored["1"] = scored["0-15"] + scored["15-30"] + scored["30-45+"]
scored["2"] = scored["45-60"] + scored["60-75"] + scored["75-90+"]
scored["ratio"] = scored["1"]/ (scored["1"] + scored["2"])
scored.sort_values(["ratio"])

,0-15,15-30,30-45+,45-60,60-75,75-90+,1,2,ratio
team,,,,,,,,,
RGN,2,2,4,7,5,9,8,21,0.275862
SD,3,4,4,10,4,6,11,20,0.354839
NJY,5,2,4,4,7,6,11,17,0.392857
WAS,3,4,4,5,4,6,11,15,0.423077
CHI,1,5,6,6,8,1,12,15,0.444444
LA,3,2,8,3,6,7,13,16,0.448276
LOU,3,4,4,5,3,4,11,12,0.478261
KC,5,7,4,5,7,4,16,16,0.500000
POR,8,9,6,5,8,8,23,21,0.522727


In [6]:
conceded = df.groupby("opponent").sum()
conceded.to_csv("data/goals_conceded_2023.csv")
conceded

,0-15,15-30,30-45+,45-60,60-75,75-90+
opponent,,,,,,
CHI,9,7,10,10,8,7
HOU,2,0,2,6,4,3
KC,5,5,8,6,5,5
LA,2,3,6,6,9,7
LOU,4,5,4,4,5,4
NC,4,3,4,3,5,6
NJY,1,4,4,6,6,2
ORL,3,6,5,3,5,5
POR,2,6,4,8,5,5


In [7]:
conceded["45-60"] + conceded["60-75"] + conceded["75-90+"]

opponent
CHI    25
HOU    13
KC     16
LA     22
LOU    13
NC     14
NJY    14
ORL    13
POR    18
RGN    10
SD     11
WAS    16
dtype: int64